In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("train.csv")
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [23]:
df["Order Date"] = pd.to_datetime(df["Order Date"], dayfirst=True, errors="coerce")
df["Ship Date"] = pd.to_datetime(df["Ship Date"], dayfirst=True, errors="coerce")
df["Sales"] = pd.to_numeric(df["Sales"], errors="coerce")
df[["Order Date", "Ship Date","Sales"]].dtypes

Order Date    datetime64[ns]
Ship Date     datetime64[ns]
Sales                float64
dtype: object

In [9]:
df.dropna(subset=["Order Date", "Sales"], inplace=True)

In [27]:
monthly_total = (
    df.resample("MS", on="Order Date")["Sales"]
      .sum()
      .reset_index()
      .rename(columns={"Order Date": "Month", "Sales": "Sales"})
)
monthly_total.head()

,Month,Sales
0,2015-01-01,14205.707
1,2015-02-01,4519.892
2,2015-03-01,55205.797
3,2015-04-01,27906.855
4,2015-05-01,23644.303


In [29]:
monthly_by_category = (
    df.groupby([pd.Grouper(key="Order Date", freq="MS"), "Category"])["Sales"]
      .sum()
      .reset_index()
      .rename(columns={"Order Date": "Month", "Sales": "Sales"})
)
monthly_by_category.head()

,Month,Category,Sales
0,2015-01-01,Furniture,6217.277
1,2015-01-01,Office Supplies,4845.140
2,2015-01-01,Technology,3143.290
3,2015-02-01,Furniture,1839.658
4,2015-02-01,Office Supplies,1071.724


In [31]:
monthly_by_region = (
    df.groupby([pd.Grouper(key="Order Date", freq="MS"), "Region"])["Sales"]
      .sum()
      .reset_index()
      .rename(columns={"Order Date": "Month", "Sales": "Sales"})
)
monthly_by_region.head()

,Month,Region,Sales
0,2015-01-01,Central,1533.966
1,2015-01-01,East,436.174
2,2015-01-01,South,9296.844
3,2015-01-01,West,2938.723
4,2015-02-01,Central,1233.174


In [33]:
df.to_csv("superstore_clean.csv", index=False)
monthly_total.to_csv("monthly_total_sales.csv", index=False)
monthly_by_category.to_csv("monthly_sales_by_category.csv", index=False)
monthly_by_region.to_csv("monthly_sales_by_region.csv", index=False)